In [1]:
import tensorflow_hub as hub

import pandas as pd
import numpy as np
import tensorflow as tf

from keras import utils
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from string import punctuation
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

import re

from sklearn import preprocessing

from nltk.stem import WordNetLemmatizer

from sklearn.metrics import classification_report

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df=pd.read_excel(r"/Revised SCOM Dump_false alert (1).xlsx")
df.head()

,Alerts,Support Queue,Ignorable (Y/N)
0,A disk cache failed to initialize,Wintel Server Support,N
1,An Array Component is in a critical Performanc...,Wintel Server Support,N
2,An error occurred in the Service Broker manager,Wintel Server Support,N
3,Application of Group Policy Alert,Wintel Server Support,Y
4,Array Performance is in a critical health state,Wintel Server Support,N


In [4]:
# counting the y/n values 
df['Ignorable (Y/N)'].value_counts()

N    136
Y     65
Name: Ignorable (Y/N), dtype: int64

In [5]:
# dropping NA in Ignorable column
df['Ignorable (Y/N)'].dropna(how='all')

0      N
1      N
2      N
3      Y
4      N
      ..
196    N
197    N
198    N
199    N
200    N
Name: Ignorable (Y/N), Length: 201, dtype: object

In [6]:
# converting Y/N to 1 and 0 
df['Ignorable (Y/N)']=df['Ignorable (Y/N)'].apply(lambda x: 1 if (x)=="Y" else 0)
df['Ignorable (Y/N)']

0      0
1      0
2      0
3      1
4      0
      ..
196    0
197    0
198    0
199    0
200    0
Name: Ignorable (Y/N), Length: 201, dtype: int64

In [7]:
# Dropping NA values in Alerts column
df['Alerts'].dropna(how='all')

0                      A disk cache failed to initialize
1      An Array Component is in a critical Performanc...
2        An error occurred in the Service Broker manager
3                      Application of Group Policy Alert
4        Array Performance is in a critical health state
                             ...                        
196      Application pool worker process is unresponsive
197    "Master Repository Update or repository replic...
198                Free Space (%) For Cluster Disk Alert
199    Cluster resource group offline or partially on...
200                    HP Hardware Component: NIC Status
Name: Alerts, Length: 201, dtype: object

In [8]:
# applying lower function to Alerts coulmn
df['Alerts'] = df['Alerts'].apply(lambda x: x.lower())
df['Alerts']

0                      a disk cache failed to initialize
1      an array component is in a critical performanc...
2        an error occurred in the service broker manager
3                      application of group policy alert
4        array performance is in a critical health state
                             ...                        
196      application pool worker process is unresponsive
197    "master repository update or repository replic...
198                free space (%) for cluster disk alert
199    cluster resource group offline or partially on...
200                    hp hardware component: nic status
Name: Alerts, Length: 201, dtype: object

In [0]:
df['Alerts'] = df['Alerts'].apply(lambda s: re.sub(r"[^a-zA-Z0-9]"," ",s))

In [0]:
lemmatizer=WordNetLemmatizer()

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens =[w for w in tokens if not w in stop_words] # [w for w in
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
    return lemmatized_output

In [11]:
# applying tokenize function to Alerts column
df['Alerts'].apply(tokenize)

0                           disk cache failed initialize
1             array component critical performance state
2                  error occurred service broker manager
3                         application group policy alert
4                array performance critical health state
                             ...                        
196         application pool worker process unresponsive
197    master repository update repository replicatio...
198                        free space cluster disk alert
199      cluster resource group offline partially online
200                     hp hardware component nic status
Name: Alerts, Length: 201, dtype: object

In [0]:
le = preprocessing.LabelEncoder()
df['Alerts'] = le.fit_transform(df['Alerts'])

In [0]:
#taking X and y
X = df['Alerts']
y = df['Ignorable (Y/N)']

In [0]:
#Scaling
scaler = StandardScaler()

In [0]:
#meta train test split
X_train_meta, X_test_meta, y_train, y_test = train_test_split((X), (y) ,stratify=y, test_size=0.20,random_state=10)

In [0]:
#scale X
standardized = scaler.fit(X_train_meta.values.reshape(-1,1))
X_train_std = standardized.transform(X_train_meta.values.reshape(-1,1))
X_test_std = standardized.transform(X_test_meta.values.reshape(-1,1))

In [0]:
embedding_128 = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"

In [0]:
#creating the embedding confuguration 

hub_layer = hub.KerasLayer(embedding_128, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [0]:
df['Support Queue'] =  df['Support Queue'].replace(to_replace='Wintel server support',value='Wintel Server Support')
# df['Support Queue'].value_counts()

In [0]:
df['Support Queue'] =  df['Support Queue'].replace(to_replace='wintel server Application',value='Wintel Server Applications')
# df['Support Queue'].value_counts()
# 

In [0]:
df['Support Queue'] = df['Support Queue'].replace(to_replace = 'citrix',value='Citrix')
# df['Support Queue'].value_counts()

In [0]:
df['Support Queue'] = df['Support Queue'].replace(to_replace = ['database , backup','Backup , Wintel server support','Wintel server support , citrix','Citrix Server Support','depends on the server name','Wintel Server Support , database','CL - Service Account Lockout','Database , Backup','backup'],value='Others')
# df['Support Queue'].value_counts()

In [0]:
df['Support Queue'] = df['Support Queue'].replace(to_replace = 'Backup ',value='Backup Support')

In [31]:
df['Support Queue'].value_counts()

Wintel Server Support         132
Wintel Server Applications     23
Database                       11
Database Support               10
Others                          9
Backup Support                  7
Storage                         4
backup                          3
Citrix                          2
Name: Support Queue, dtype: int64

In [0]:
X_supportQueue = df[['Support Queue']]

In [0]:
X_train_nlp, X_test_nlp, y_train, y_test = train_test_split(X_supportQueue, (y) ,stratify=y, test_size=0.20,random_state=10)


In [0]:
###Train Data
embed = (hub_layer(X_train_nlp.values.reshape(-1,)))
embed_df = pd.DataFrame((embed.numpy()[:].reshape(-1,128)))
train_std_df = pd.DataFrame(X_train_std)
concat_df = pd.concat([train_std_df,embed_df] ,axis=1)

In [0]:
##Test Data
embed_test = (hub_layer(X_test_nlp.values.reshape(-1,)))
embed_test_df = pd.DataFrame((embed_test.numpy()[:].reshape(-1,128)))
test_std_df = pd.DataFrame(X_test_std)
concat_test_df = pd.concat([test_std_df,embed_test_df] ,axis=1)

In [0]:
input2 = tf.keras.layers.Input(shape=(129,))
dense_layer_1 = tf.keras.layers.Dense(32, activation='relu')(input2)
drop1= tf.keras.layers.Dropout(.2)
dense_layer_2 = tf.keras.layers.Dense(32, activation='relu')(dense_layer_1)
drop1= tf.keras.layers.Dropout(.2)
dense_layer_3 = tf.keras.layers.Dense(32, activation='relu')(dense_layer_2)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense_layer_3)
model = tf.keras.Model(inputs=input2, outputs=output)

In [37]:
model.summary()

model.compile(optimizer='Adam',
              #loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 129)]             0         
_________________________________________________________________
dense (Dense)                (None, 32)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________


In [0]:
from tensorflow.keras.utils import to_categorical

In [39]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(concat_df.values, y_train.values,
                    epochs=100,
                    validation_data=(concat_test_df.values, y_test.values),
                    verbose=1,callbacks=[callback])

Epoch 1/100
5/5 [==============================] - 0s 38ms/step - loss: 0.6882 - accuracy: 0.5625 - val_loss: 0.6743 - val_accuracy: 0.6829
Epoch 2/100
5/5 [==============================] - 0s 7ms/step - loss: 0.6691 - accuracy: 0.6750 - val_loss: 0.6588 - val_accuracy: 0.6829
Epoch 3/100
5/5 [==============================] - 0s 7ms/step - loss: 0.6561 - accuracy: 0.6750 - val_loss: 0.6471 - val_accuracy: 0.6829
Epoch 4/100
5/5 [==============================] - 0s 8ms/step - loss: 0.6433 - accuracy: 0.6750 - val_loss: 0.6404 - val_accuracy: 0.6829
Epoch 5/100
5/5 [==============================] - 0s 7ms/step - loss: 0.6380 - accuracy: 0.6750 - val_loss: 0.6348 - val_accuracy: 0.6829
Epoch 6/100
5/5 [==============================] - 0s 8ms/step - loss: 0.6332 - accuracy: 0.6750 - val_loss: 0.6285 - val_accuracy: 0.6829
Epoch 7/100
5/5 [==============================] - 0s 7ms/step - loss: 0.6306 - accuracy: 0.6750 - val_loss: 0.6235 - val_accuracy: 0.6829
Epoch 8/100
5/5 [=========